# Capstone Project Notebook
This notebook contains the capstone project work done for the IBM Professional Certificate on Data Science. It will be updated as more details emerge.

On this notebook, we are collecting the neighborhood data from Wikipedia, and then adding the geo-coordinates. My first step is to define the libraries that I will be using.

In [2]:
import pandas as pd
import numpy as np
print('Libraries imported.')

Libraries imported.


## Getting the data and pre-processing

Getting the data required some research. I used the [trick](https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/kBaFtPNGSHGWhbTzRohxtQ) by [Mutlu Okumus](https://www.coursera.org/learn/applied-data-science-capstone/profiles/0b18aae3b5eabac80cc71c57ba7f02b8), whereby I collected the data from a previous version of the page. Then, I used plain Pandas to scrap the data.

A few things that I also noticed from the process:

1. There were no 'Not assigned' Neighbourhood values that had a Borough different from 'Not assigned'.
2. In general, a postcode is always within one borough

The steps are detailed as comments in the code.

In [3]:
# Importing the data from web using pandas.
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379'
data_raw = pd.read_html(url)
# The table will be located in the first element of the series
data_tbl = data_raw[0]
# I get rid of any element which has a 'Borough' equal to 'Not assigned'
data_tbl = data_tbl.loc[data_tbl['Borough'] != 'Not assigned']
# I pull out the unique postcode values
postcode = data_tbl['Postcode'].value_counts().index
# Create an empty dataframe to which all the data will be attached
df = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
i = 0

# For every single unique postcode
for pc in postcode:
    # Extract the data for that unique postcode
    idx = data_tbl['Postcode'] == pc
    data_pc = data_tbl.loc[idx][['Borough','Neighbourhood']]
    # We assume that a postcode is always within a unique Borough
    borough = data_pc['Borough'].value_counts().index[0]
    # Concatenate the neighborhoods into one string
    neigh = data_pc['Neighbourhood'].str.cat(sep=", ")
    # Now, we add a new line to our dataframe
    df.loc[i] = [pc, borough, neigh]
    # Make sure that we move to the next line
    i = i+1;
# Our data frame is complete, we sort the result by postcode
df.sort_values(by=['Postcode'], inplace=True)
# Clean up the indexes so they appear in order
df.index = range(0,i)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Unfortunately, using the geocoder package was too time-consuming, with one single postcode (out of 103), taking a significant amount of time. I guess it is being heaviliy used. Therefore, I moved to use the 'Geospatial_Coordinates.csv' provided in the instructions. Assuming that I got the correct neighborhood data, I will sort this data by 'postcode' too, just to be sure that I have the right order on the rows.

In [4]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.sort_values(by=['Postal Code'], inplace=True)
df_coord.index = range(0,i)
df_coord.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


With that out of the way, I will just concatenate the 'Latitude' and 'Longitude' columns to my original dataframe.

In [5]:
df = df.join(df_coord[['Latitude','Longitude']])
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Setting up the geographical data and first maps

Now, it is time to do the clustering. In essence, I will replicate the analysis done in the lab for simplicity, and because I don't have too much time to think this too thoroughly.
Again, as first step, we need to load the necessary libraries.

In [6]:
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
from sklearn.metrics import silhouette_score
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


As in the lab example, we define an instance of the geocoder. First, we define a user_agent, which I will name 'explorer'.

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let's have a look at a map. Using the same code as in the lab, we have a wide area view of the Toronto metropolitan area.

In [29]:
# create map of Toronto using latitude and longitude values
map_toronto_metro = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_metro)  
    
map_toronto_metro

That was succesful. I will follow the suggestion in the instructions of only process the neighborhoods within the boroughs of Toronto, as I suspect that later on, the number of queries to the Foursquare API would make it a necessity.

In [9]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


That was painless. Let us redraw the map. I won't reorganize the coordinates, as I suspect that they are the same.

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Collecting data from Foursquare

OK, we have gotten here so far. Here is where I am not sure how to make this hidden. It seems to me rather dangerous to provide my keys openly. I will see if this helps or not.

In [11]:
# @hidden_cell
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

I will skip the exploration of one neighborhood, and focus on the bigger picture. I will translate the function using to batch process the data

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's apply this operation to our Neighborhood data.

In [13]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                 latitudes=toronto_df['Latitude'],
                                 longitudes=toronto_df['Longitude'])
print(toronto_venues.shape)
toronto_venues.head(10)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,Moksha Yoga Danforth,43.677622,-79.352116,Yoga Studio


I am going to skip most of the Lab's analysis and focus on the key data processing steps. The gist of the lab is to identify the type of 'venues' that are most common on a given cluster. For this purpose, the data above is split into dummy variables to check the frequency of the particular venues. I will copy this analysis, and make the proper changes so it works with my data.

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Now we see the frequency data for each type of venue in Toronto
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(10)

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.0,...,0.0,0.0,0.020000,0.0,0.000000,0.000000,0.000000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.017857,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.043478
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.0625,0.125,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.022222,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012987,0.0,...,0.0,0.0,0.012987,0.0,0.000000,0.012987,0.000000,0.000000,0.00,0.012987
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.037975,0.0,0.063291,0.012658,0.000000,0.000000,0.00,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.012048,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012048,0.0,...,0.0,0.0,0.000000,0.0,0.012048,0.000000,0.012048,0.012048,0.00,0.024096


The next step is just to identify the top 10 venues. I'm copying the function used to do so in the Lab.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

This will return the most common venues in a dataframe.

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Café,Bar,Bakery,Office,Pizza Place,Clothing Store,Hotel,Sushi Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Shopping Mall
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Airport Service,Coffee Shop,Rental Car Location
5,"Cabbagetown, St. James Town",Coffee Shop,Park,Restaurant,Café,Pub,Italian Restaurant,Chinese Restaurant,Bakery,Pizza Place,Sandwich Place
6,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Thai Restaurant,Japanese Restaurant,Burger Joint,Café,Spa,Middle Eastern Restaurant,Ice Cream Shop
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Grocery Store,Burger Joint,Bakery
8,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Candy Store,Restaurant,Diner,Nightclub,Baby Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Mediterranean Restaurant,Hotel,Gastropub,Café,Yoga Studio


## Clustering the neighborhoods

It is time to use k-means to identify similar neighbourhoods. In essence I follow the procedure given in the lab, but I will use Silhouette analysis to determine the correct value of k. The value of the silhouette score ranges from -1 (worst) to 1 (best), and the objective is to maximize the score.

In [17]:
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

k_vals = range(2,11)
silhouette_avg = np.zeros(len(k_vals))

for i in range(0,len(k_vals)):
    # run k-means clustering
    kmeans = KMeans(n_clusters=k_vals[i], random_state=0)
    cluster_labels = kmeans.fit_predict(toronto_grouped_clustering)
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg[i] = silhouette_score(toronto_grouped_clustering, cluster_labels)
    print("For n_clusters =", k_vals[i], "The average silhouette_score is :", round(silhouette_avg[i],4))


For n_clusters = 2 The average silhouette_score is : 0.6491
For n_clusters = 3 The average silhouette_score is : 0.5403
For n_clusters = 4 The average silhouette_score is : 0.3842
For n_clusters = 5 The average silhouette_score is : 0.4181
For n_clusters = 6 The average silhouette_score is : 0.4154
For n_clusters = 7 The average silhouette_score is : 0.0815
For n_clusters = 8 The average silhouette_score is : 0.084
For n_clusters = 9 The average silhouette_score is : 0.0639
For n_clusters = 10 The average silhouette_score is : 0.1003


The maximum silhouette value is given for k = 2. However, this may be a very small value. I will take 3, as there is an inmediate drop at 4.

In [18]:
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 4, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

I decided to separate this merge, as it causes problems in the line below while I was experimenting with it.

In [19]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Now, I can examine the most common vanues for each neighborhood.

In [20]:
# add clustering labels
toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Trail,Pub,Health Food Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Liquor Store,Spa,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Gym,Park,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Coffee Shop,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Park,Bus Line,Swim School,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Food & Drink Shop,Breakfast Spot,Park,Department Store,Hotel,Sandwich Place,Dog Run,Doner Restaurant,Dumpling Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Seafood Restaurant,Spa,Fast Food Restaurant,Salon / Barbershop,Mexican Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Asian Restaurant,Pub
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Tennis Court,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Coffee Shop,Pub,Light Rail Station,American Restaurant,Bank,Restaurant,Fried Chicken Joint,Sports Bar,Supermarket,Sushi Restaurant


Let's create the final map with the clusters being color coded.

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Broadly speaking, we can separate the neighbourhoods of Toronto into five classes. The first one of them seems to be an entretainment area, with plenty of cafes and restaurants.

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Neighborhood,Trail,Pub,Health Food Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Liquor Store,Spa,Brewery,Bubble Tea Shop
2,"The Beaches West, India Bazaar",0,Sandwich Place,Gym,Park,Brewery,Burrito Place,Restaurant,Pub,Pizza Place,Coffee Shop,Movie Theater
3,Studio District,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place
5,Davisville North,0,Gym,Food & Drink Shop,Breakfast Spot,Park,Department Store,Hotel,Sandwich Place,Dog Run,Doner Restaurant,Dumpling Restaurant
6,North Toronto West,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Seafood Restaurant,Spa,Fast Food Restaurant,Salon / Barbershop,Mexican Restaurant
7,Davisville,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Asian Restaurant,Pub
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,Light Rail Station,American Restaurant,Bank,Restaurant,Fried Chicken Joint,Sports Bar,Supermarket,Sushi Restaurant
11,"Cabbagetown, St. James Town",0,Coffee Shop,Park,Restaurant,Café,Pub,Italian Restaurant,Chinese Restaurant,Bakery,Pizza Place,Sandwich Place
12,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Mediterranean Restaurant,Hotel,Gastropub,Café,Yoga Studio


The second seems to be dominated by jewlery stores, which makes me think that there is a concentration of this type of busines in this area. It is probably affluent.

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Forest Hill North, Forest Hill West",1,Jewelry Store,Mexican Restaurant,Trail,Sushi Restaurant,Yoga Studio,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


The third class seems to be a park area.

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Photography Studio,Park,Bus Line,Swim School,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Rosedale,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


The fourth class seems to be dominated by a major music venue, surounded by restaurants and cafes.

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,3,Music Venue,Garden,Yoga Studio,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


The last area seems to have a high concentration of tennis courts, which may indicate a large sports area is nearby.

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",4,Tennis Court,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


That's all. Thanks!